In [2]:
# Model Training and Evaluation

# Retrieve previously stored variables
%store -r x_train # train samples: 2185
%store -r x_test # testing samples: 547
%store -r y_train # categories train: 2185
%store -r y_test # 
%store -r yy # total samples
%store -r le

In [36]:
display(x_train[1])
print(len(y_train))
display(len(yy))
print(len(x_test))
print(x_train.shape)

array([-3.19514862e+02,  1.39700089e+02, -1.12087608e+02, -4.92915726e+01,
       -1.74340093e+00, -5.69976540e+01, -4.37130013e+01, -6.43806553e+00,
       -1.52856960e+01, -2.53208313e+01, -1.17009325e+01, -7.51425409e+00,
       -9.69702148e+00, -2.19402933e+00,  7.07967758e+00,  1.73335800e+01,
        1.20680609e+01, -1.13923674e+01, -1.51345863e+01, -3.63308001e+00,
        3.87587762e+00,  1.19124365e+01,  5.14146280e+00, -9.87630939e+00,
       -6.61446285e+00,  2.29413822e-01, -5.05282927e+00, -3.69123459e+00,
        7.10211515e+00,  8.15948486e+00,  2.64397812e+00,  9.79319692e-01,
       -1.59480238e+00, -1.82388520e+00,  2.49361086e+00,  5.21429539e+00,
        1.82872808e+00, -6.47891617e+00, -5.83984756e+00,  2.10689330e+00],
      dtype=float32)

2185


2732

547
(2185, 40)


In [37]:
# Import libraries
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn import metrics 

# Extract the number of labels - 4 in our case
num_labels = yy.shape[1]

# Begin constructing ML model
# Create an object of the Sequential class
model = Sequential()

# Create input layer using the Dense function
numNodes = 64; # Trial and error - 1/4 of 256 bit
numMFCC = 40;
# Input shape is the size of the input array (1-D array of 40 columns, 1 row)
model.add(Dense(numNodes, input_shape=(numMFCC,)));
# Specifying the activation function to be used - relu: Rectified Linear Activiation
model.add(Activation('relu'))
# Dropout value of 50% - means random half of neurons exluded from each update cycle. Used to prevent overfitting.
model.add(Dropout(0.5))


# Add subsequest hidden layer - DENSE function
model.add(Dense(numNodes))
model.add(Activation('relu'))
model.add(Dropout(0.5))

# Add the output layer - DENSE function
# Output nodes is the different categories
# Different usage of activation function
model.add(Dense(num_labels))
model.add(Activation('softmax'))

Using TensorFlow backend.


In [38]:
# Compile the model
# Optimizer "adam" is a typical optimizer used - variation SGD (stochastic gradient descent)
# SGD utilizes the gradient of the loss function with respects to the weight
# loss -> typical loss function 
# metrics is output to be displayed (accuracy is the output of the loss function (?))
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

In [39]:
# Display model architecture summary 
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 64)                2624      
_________________________________________________________________
activation_1 (Activation)    (None, 64)                0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
activation_2 (Activation)    (None, 64)                0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 4)                